In [1]:
import json
import requests
import urllib.request
import time
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
df = pd.DataFrame()
index = 0
last_date = ''

# 수정 변수들
coinName = "BTC"
to_date = '2021-06-21'
num = 30
scale = '1'
if num == 0:
    type = 'days'
else:
    type = 'minutes/{}'.format(num)


def get_coin_data_url(coinName, type, scale, cnt=400) :
    addr = 'https://crix-api-endpoint.upbit.com/v1/crix/candles/'
    if type == 'minutes' :
        basic_url = addr + type + '/' + scale + '?code=CRIX.UPBIT.KRW-' + coinName +'&count='+str(cnt)
    else :
        basic_url = addr + type + '/' + '?code=CRIX.UPBIT.KRW-' + coinName +'&count='+str(cnt)

    return basic_url

basic_url = get_coin_data_url(coinName,type,scale)
url = basic_url

def get_upbit_data(url, last_date, to_date) :
    global df
    global index

    fail2GetData = False
    response = ''
    while True:
        try :
            response = requests.get(url)
        except Exception as e:
            time.sleep(20)
            continue
        if str(response) == '<Response [200]>':
            break
        time.sleep(10)
    if ( fail2GetData )  :
        exit()

    data = response.json()

    if index == 0:
        df = pd.DataFrame(data)
        date = ''
        date = data[len(data)-1]['candleDateTime']
        index +=1

    else:
        df_for = pd.DataFrame(data)
        df = df.append(df_for,ignore_index=True)
        date = ''
        date = data[len(data)-1]['candleDateTime']
    return date




In [2]:

while (1) :
    last_date = get_upbit_data(url, last_date, to_date)
    tmp1 = last_date.split('T')
    if tmp1[0]  < to_date :
        break
    tmp2 = tmp1[1].split('+')
    target_date = tmp1[0] + ' ' + tmp2[0]
    url = basic_url + '&to=' + target_date    #to=2019-11-27 04:01:00
    time.sleep(2)

idx_todate = df.index[(df.candleDateTimeKst == '{}T00:00:00+09:00'.format(to_date))].tolist()[0]+1
idx_todate

df = df.drop(df.index[idx_todate:])

In [3]:
df['Date'] = 0
df['hour'] = 0
df['minute'] = 0

In [4]:
for i in range(len(df)):
    df['Date'] [i] = df['candleDateTimeKst'][i].split('T')
    df['hour'] [i] = df['Date'][i][1].split('+')[0]
    df['minute'] [i] = df['hour'][i].split(':')[1]
    df['hour'] [i] = df['hour'][i].split(':')[0]
    df['Date'] [i] = df['Date'][i][0]


In [5]:
drops= ['Date','unit','candleAccTradePrice','openingPrice','code','candleDateTime','timestamp','candleDateTimeKst']
df=df.drop(drops,axis=1)
df=df[['hour','minute','tradePrice','highPrice','lowPrice','candleAccTradeVolume']]
df.head()

,hour,minute,tradePrice,highPrice,lowPrice,candleAccTradeVolume
0,16,0,36607000.0,37266000.0,36522000.0,615.408155
1,15,30,37146000.0,37732000.0,37100000.0,357.372051
2,15,0,37666000.0,37839000.0,37626000.0,124.219095
3,14,30,37689000.0,37750000.0,37558000.0,130.206013
4,14,0,37737000.0,37999000.0,37629000.0,282.300025


In [6]:
ups, downs = df['tradePrice'].loc[::-1].diff() ,df['tradePrice'].loc[::-1].diff()
ups[ups<0] =0
downs[downs>0] = 0
period = 14 
au = ups.abs().ewm(com = period-1, min_periods = period).mean() 
ad = downs.abs().ewm(com = period-1, min_periods = period).mean() 
RS = au/ad 
RSI = pd.Series(100 - (100/(1+RS))) 
RSI = RSI.loc[::-1]
print(RSI)
df['RSI'] = RSI

0      31.955811
1      39.345824
2      49.627014
3      50.165394
4      51.242680
         ...    
268          NaN
269          NaN
270          NaN
271          NaN
272          NaN
Name: tradePrice, Length: 273, dtype: float64


In [7]:
# CCI 오예
N = 20
D = 0
m = 0
for i in range(3):
    D = 0
    m = 0
    sm= 0

    M = ((df['tradePrice'][i] + df['lowPrice'][i] + df['highPrice'][i]) / 3)

    for j in range(N):
        jM = (df['tradePrice'][i+j] + df['lowPrice'][i+j] + df['highPrice'][i+j]) / 3
        sm +=jM
    sm = (sm/N)

    for k in range(N):
        kM = (df['tradePrice'][i+k] + df['lowPrice'][i+k] + df['highPrice'][i+k]) / 3
        D += abs(sm-kM)
    D = D/N
    print('D : {}  // M : {} // m : {}'.format(D,M,sm))

    CCI = (M - sm) / (D * 0.015)

    print(CCI)


D : 181276.66666666605  // M : 36798333.333333336 // m : 37370433.33333333
-210.3966313002205
D : 155311.66666666605  // M : 37326000.0 // m : 37399283.33333333
-31.45646209592697
D : 169526.66666666642  // M : 37710333.333333336 // m : 37418966.666666664
114.58046587230015


In [11]:
s_data, l_data = df['tradePrice'].loc[::-1],df['tradePrice'].loc[::-1]
s_data
l_data

272    40830000.0
271    40838000.0
270    40938000.0
269    40870000.0
268    41017000.0
          ...    
4      37737000.0
3      37689000.0
2      37666000.0
1      37146000.0
0      36607000.0
Name: tradePrice, Length: 273, dtype: float64

In [18]:
# MACD
period = 12
s_data, l_data = df['tradePrice'].loc[::-1],df['tradePrice'].loc[::-1]
# s_data, l_data = df['tradePrice'].diff() ,df['tradePrice'].diff()
a12 = s_data.abs().ewm(com = period-1, min_periods = period).mean() 
a12.head(15)


272             NaN
271             NaN
270             NaN
269             NaN
268             NaN
267             NaN
266             NaN
265             NaN
264             NaN
263             NaN
262             NaN
261    4.138548e+07
260    4.144915e+07
259    4.148463e+07
258    4.150365e+07
Name: tradePrice, dtype: float64

In [19]:
period = 26
a26 = l_data.abs().ewm(com = period-1, min_periods = period).mean() 
a26

272             NaN
271             NaN
270             NaN
269             NaN
268             NaN
           ...     
4      3.810330e+07
3      3.808737e+07
2      3.807116e+07
1      3.803558e+07
0      3.798063e+07
Name: tradePrice, Length: 273, dtype: float64

In [20]:
macd = a12-a26
macd

272              NaN
271              NaN
270              NaN
269              NaN
268              NaN
           ...      
4     -494569.603253
3     -471945.665910
2     -451524.075833
1     -455409.949021
0     -481560.777130
Name: tradePrice, Length: 273, dtype: float64

In [22]:
def EMA(data, period=20,column='Close'):
    return  data[column].emw(span=period,adjust=False).mean()

def MACD(data,period_long=26,period_short=12,period_signal=9, column='Close'):
    ShortEMA = EMA(data,period_short,column=column)
    LongEMA = EMA(data,period_long,column=column)

    data['MACD'] = ShortEMA,LongEMA
    data['Signal_Line'] = EMA(data, period_signal, column='MACD')

    return data
df['Close'] = df['tradePrice']
df = MACD(df,period_long=26,period_short=12,period_signal=9)
df

AttributeError: 'Series' object has no attribute 'emw'